In [ ]:
import os
import skimage.io as io
import skimage.transform as trans
import random
import h5py
import re
import matplotlib
import rasterio
import numpy as np
from rasterio.plot import show, show_hist
from rasterio.merge import merge as rastermerge
from matplotlib import pyplot
%matplotlib inline
from keras import metrics
from keras import backend as K
from keras.initializers import *
from keras.layers import *
from keras.models import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from importlib import reload
from keras.callbacks import EarlyStopping, TensorBoard
from keras.models import load_model
from keras.models import save_model

In [ ]:
PREDICTIONS_DIR = "/Users/ciaran/data/imagery/tifs/blue_anchor/predictions"
#IMAGES_TO_PREDICT_DIR = "/Users/ciaran/data/imagery/tifs/blue_anchor/train/normalised_images"
IMAGES_TO_PREDICT_DIR = "/Users/ciaran/data/imagery/tifs/blue_anchor/test_pred"

In [ ]:
def sorensen_dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    coef = (2. * intersection + K.epsilon()) / (K.sum(y_true_f) + K.sum(y_pred_f) + K.epsilon())
    return coef

def sorensen_dice_coef_loss(y_true, y_pred):
    return 1 - sorensen_dice_coef(y_true, y_pred)

In [ ]:
MODEL = load_model('/Users/ciaran/dev/dsa_project/models/1e-5_subset_2/model')

# Make Predictions and Rasterize

In [ ]:
predictedImages = [file for file in os.listdir(PREDICTIONS_DIR)]

In [ ]:
def updateImageMetadata(metadata):
    meta = metadata[0]
    meta.update({"driver":"GTiff",
                 "height": metadata[1][0],
                 "width": metadata[1][1],
                 "transform": metadata[3],
                 "crs": metadata[2],
                 "count": 1,
                 "dtype": 'float32'})
    return meta

def writePredictionAsGeoTIFF(prediction, imageToPredictMetadata, file):
    print(imageToPredictMetadata)
    updatedImageMetadata = updateImageMetadata(imageToPredictMetadata)
    print(updatedImageMetadata)
    with rasterio.open(os.path.join(PREDICTIONS_DIR, file), "w", **updatedImageMetadata) as dest:
                    dest.write(prediction[0][:])

def predictImage(image):
    prediction = MODEL.predict(x=np.array([image]))
    return prediction

def getImageToPredictsMetadata(path):
    image = rasterio.open(path)
    return [image.meta, image.shape, image.crs, image.transform]

def getImageToPredict(path):
    return io.imread(path)

def predictAndWriteResultOfFolderImages(path):
    for file in os.listdir(path):
            if file.endswith(".tif") and file not in predictedImages:
                imageToPredict = getImageToPredict(os.path.join(path, file))
                imageToPredictMetadata = getImageToPredictsMetadata(os.path.join(path, file))
                prediction = predictImage(imageToPredict)
                print(prediction.shape)
                writePredictionAsGeoTIFF(prediction, imageToPredictMetadata, file)
                show(prediction)
                imageToPredict = None
                imageToPredictMetadata = None
                prediction = None
                print("Processed: " + file)

In [ ]:
predictAndWriteResultOfFolderImages(IMAGES_TO_PREDICT_DIR)

# Merge images

In [ ]:
def generateBeachFromCells(pathToBeachImages, nameOfBeach):
    imagesToMerge = getAllImagesToMerge(pathToBeachImages)
    mergedImage, mergedImageTransform = rastermerge(imagesToMerge)
    metadata = imagesToMerge[0].meta.copy()
    crs = imagesToMerge[0].crs
    writeImageAsGeoTIFF(mergedImage, mergedImageTransform, metadata, crs, nameOfBeach)

def getAllImagesToMerge(pathToBeachImages):
    return [rasterio.open(os.path.join(pathToBeachImages, image)) for image in os.listdir(pathToBeachImages)]

def writeImageAsGeoTIFF(img, transform, metadata, crs, filename):
    metadata.update({"driver":"GTiff",
                     "height":img.shape[1],
                     "width":img.shape[2],
                     "transform": transform,
                     "crs": crs,
                     "nodata": 0})
    with rasterio.open(filename+".tif", "w", **metadata) as dest:
        dest.write(img)

In [ ]:
generateBeachFromCells("/Users/ciaran/data/imagery/tifs/blue_anchor/predictions", "mosaic")